### Demo: Graph DB and Topological features
- This notebook generates the graph-based features that is updated on a daily bases
- The notebook also shows how we add notes to the graph database.
- At the end of this notebook, we concat all computed features (in the two notebooks). 
- **Note** Run notebook `0_genfeats_nongraph.ipynb` before this one, as the graph data depends on some files generated from that notebook

In [1]:
import pandas as pd
import numpy as np
import datetime
import os

### Graph DB and Topological features
---
This notebook shows how we add nodes into neo4j database and compute graphical features:
- We provide a dummy dataset:
    - `dummy100.csv`: we sampled 100 rows of non-user-sensitive periodic FQDNs from 2021-03-30. We carefully examined every host name to ensure there's no privacy leakage. We restricted FQDNs to domains that have periodic activity because non-periodic domains are filtered out before feature generation in daily pipeline.
    - The file contains the host name, server IP and port extracted from Zeek logs. Client IPs are faked to avoid privacy leakage.
    - `periodic100.parquet`: the corresponding periodicity detection results of the 100 servers.
    - `hist100.parquet`: the history file we just computed in `0_genfeats_nongraph.ipynb`.
    - `cisco_top1m.csv`: the cisco top 1 million data we pulled on 2021-03-30.
    - `malicious_hist.csv`: this sampled file keeps track of historical malicious servers.
- The below code reads the above dummy data:

In [2]:
host100 = pd.read_csv("dummydata/dummy100.csv")
per100 = pd.read_parquet("dummydata/periodic100.parquet")
cisco_1m = pd.read_csv("dummydata/cisco_top1m.csv", names=["rank", "host"])
hist100 = pd.read_parquet("dummydata/hist100.parquet") 
malhist = pd.read_csv("dummydata/malicious_hist.csv")

### [Important] Neo4j GraphDB
----
- If you have configured the graph database as described in the README.md, skip the following installation guidance.

#### Neo4j GraphDB Installation and Configuration
- Before we start to compute topological features, we need to install and configure graph database. The following code was deployed on neo4j graph database community version 4.4.0
- Offical installation guidance can be found at: https://debian.neo4j.com/?_ga=2.50993706.2051114848.1695324818-590444767.1695324818

- To install neo4j 4.4.0:
    - `wget -O - https://debian.neo4j.com/neotechnology.gpg.key | sudo apt-key add -`
    - `echo 'deb https://debian.neo4j.com stable 4.4' | sudo tee /etc/apt/sources.list.d/neo4j.list`
    - `sudo apt-get update`
    - `sudo apt-get install neo4j=1:4.4.0`
- After installation complete, check the installation status in the command line: 
    - `neo4j status`
    - if you see something like `neo4j is not running`, then we have a successful installation.
- Next we need to start the graph db:
    - `sudo neo4j start`
- After we start the neo4j database, you need to change the default password of neo4j. Type command: 
    - `sudo cypher-shell`
    - If you see `connection refused`, wait another 20 seconds and retry. There is a short delay after starting the graph db.
    - The default username is:`neo4j`, and the default password is:`neo4j`. Change the password to your own password.
    - If you are using our test server, the default password is set to `acsac`.
- Leave the `cypher-shell` using `Ctrl + D`
- **Important.** We need to config the database configuration before we start the database. There are two approaches:
    - **First approach**: 
        - you can copy our provided configuration to rewrite the configurate.
        - `sudo mv /etc/neo4j/neo4j.conf /etc/neo4j/neo4j_bk.conf`
        - `sudo cp /home/ubuntu/bcndetection/dummypipeline/neo4j.conf /etc/neo4j/neo4j.conf`
    - **Second approach**: 
        - you can directly edit the file by navigating to the default config file at `/etc/neo4j/neo4j.conf`. 
        - You need **sudo** to edit this file, e.g. `sudo vim /etc/neo4j/neo4j.conf`
        - **Change:** `dbms.directories.import=$path_to_repo$/dummypipeline/dummydata/graphtmp`
        - For example: `dbms.directories.import=/home/ubuntu/bcndetection/dummypipeline/dummydata/graphtmp`
        - Next, we need to **add** the following `dbms.security.procedures.unrestricted=apoc.*` in the neo4j.conf
        - **Save the configuration file.**
- **Important.** Install Neo4j APOC Core functions:
    - `sudo mv /var/lib/neo4j/labs/apoc-*core.jar /var/lib/neo4j/plugins/`
- **Important.** Configure Neo4j APOC function:
    - copy the provided configuration file to the same neo4j configuration folder:
    - `sudo cp /home/ubuntu/bcndetection/dummypipeline/apoc.conf /etc/neo4j/apoc.conf`
- **Important.** We need to restart the database after the configuration with command: `sudo neo4j restart`
- By default, you can visualize the graphdb at: `http://localhost:7474/browser/`

- Run the below code to add nodes into the database and compute related features

In [3]:
import tldextract
from neo4j import GraphDatabase
from src.addnode import AddNode
from src.temporal_feats import merge_cisco

In [4]:
def prepare_graph_inputdata(logdf, ciscodf, logday, fqdn_col="host"):
    logdf = merge_cisco(logdf, ciscodf)
    logdf["logday"] = logday
    logdf["domain"] = logdf[fqdn_col].apply(lambda x: parse_reg_dom(x))
    logdf["isIP"] = logdf[fqdn_col].apply(lambda x: isIP(x))
    return logdf

    
def parse_reg_dom(text):
    try:
        ext = tldextract.extract(text)
        if ext.registered_domain == '':
            return text
        return ext.registered_domain
    except:
        return text
    

def isIP(dom_text):
    """check if string is an ip address"""
    try:
        _ = ipaddress.ip_address(dom_text)
        return 1
    except:
        return 0

#### Path configuration and preparation

In [5]:
logday = "2021-03-30"
data_fmt = "data_{}.csv"
feature_dir = "dummydata/features/{}".format(logday)

# set tmp dir for graph db to read and write
graphtmpdir = "dummydata/graphtmp/" 
# set the relative path, this value is defined by the $dbms.directories.import in the neo4j.conf
graphimport_rel = "file:///" 

# we first prepare the data that we want to add to the graphdb
logdf = host100.drop(columns=["id_orig_h"])
graphdf = prepare_graph_inputdata(logdf, cisco_1m, logday)

# next we save it to graphtmp folder
# Neo4j can directly read files in the import folder specified by $dbms.directories.import
data_fpath = os.path.join(graphtmpdir, data_fmt.format(logday))
print("save data to graph db tmp dir:", data_fpath)
graphdf.to_csv(data_fpath, index=False)

# next we set up fpath for data, historical malicious files, and features
graph_relfpath = os.path.join(graphimport_rel, data_fmt.format(logday))
print("relative fpath for data:", graph_relfpath)
histmal_relfpath = os.path.join(graphimport_rel, "malicious_hist.csv")
print("relative fpath for historical malicious file:", histmal_relfpath)


save data to graph db tmp dir: dummydata/graphtmp/data_2021-03-30.csv
relative fpath for data: file:///data_2021-03-30.csv
relative fpath for historical malicious file: file:///malicious_hist.csv


### Next we perform a series of computation in graph db
- **Important** make sure the graphdb is started before run the following cells.
    - to check the neo4j status: `sudo neo4j status`
    - to start the neo4j db: `sudo neo4j start` 
    - wait for ~30 seconds before running the cell, if the graph db is just started. 
- we add FQDN, Domain, IP and corresponding relationships.
- we update the graph db using historical malicious files.
- we compute graph-related features and save them in files.

#### Change the password in the following cell to connect to graphdb
- if you are using our provided testing server, the username is neo4j and the password is acsac

In [6]:
### change the password (third argument) to your plaintext password
dbhandler = AddNode("neo4j:anon_resphost:7687", "neo4j", "acsac")

In [7]:
### add FQDN, Domain, IP and corresponding relationships
dbhandler.add_nodes(graph_relfpath)

### update the graph db using historical malicious files.
dbhandler.update_labels(histmal_relfpath)

## we compute graph-related features.
## both function compute statistics from neighboring nodes
dbhandler.update_domMal_feats()
dbhandler.update_ipMal_feats()

------Adding Nodes------
------Finish Adding Nodes------
<Record count(f)=100>
----------Updating GraphDB Lables---------
----------Finish Updating GraphDB Lables---------
<Record count(f)=5437>
----------Updating GraphDB Domain Features---------
----------Finish Updating GraphDB Domain Features---------
<Record count(d)=88>
----------Updating GraphDB IP Features---------
----------Finish Updating GraphDB IP Features---------
<Record count(i)=98>


In [8]:
## we save the computed featues to files
## absolute path here
domscore_fpath = os.path.join("/home/ubuntu/bcndetection/dummypipeline/dummydata/features/2021-03-30", "domscore_raw.csv")
ipscore_fpath = os.path.join("/home/ubuntu/bcndetection/dummypipeline/dummydata/features/2021-03-30", "ipscore_raw.csv")

dbhandler.domscore_to_csv(domscore_fpath)
dbhandler.ipscore_to_csv(ipscore_fpath)

----------Writing GraphDB Domain Features---------
----------Finish Writing GraphDB Domain Features---------
<Record file='/home/ubuntu/bcndetection/dummypipeline/dummydata/features/2021-03-30/domscore_raw.csv' source='statement: cols(15)' format='csv' nodes=0 relationships=0 properties=1500 time=7 rows=100 batchSize=20000>
----------Writing GraphDB IP Features---------
----------Finish Writing GraphDB IP Features---------
<Record file='/home/ubuntu/bcndetection/dummypipeline/dummydata/features/2021-03-30/ipscore_raw.csv' source='statement: cols(15)' format='csv' nodes=0 relationships=0 properties=1470 time=5 rows=98 batchSize=20000>


In [9]:
## Close the database
dbhandler.close()

### Compute graph-based features
- given the raw score, we compute graph based features:
    - domain-related features 
    - ip-related features
    - distance to nearest malicious neighbors

In [10]:
from src.graphscore_feats import gen_dom_graphscore, gen_ip_graphscore, gen_len2mal_score

In [11]:
domscore = gen_dom_graphscore(logday, datafpath=data_fpath, 
                              savefpath=os.path.join(feature_dir, "features_domgraph.parquet"),
                              rawfpath=os.path.join(feature_dir, "domscore_raw.csv"))

[Info]====== Generate Domain Graph Features for date: 2021-03-30 ======
[Info] Raw Data shape: (100, 2)
(100, 18)
[Info] Graph Domain Features Save Data to: dummydata/features/2021-03-30/features_domgraph.parquet


In [12]:
ipscore = gen_ip_graphscore(logday, datafpath=data_fpath,
                            savefpath=os.path.join(feature_dir, "features_ipgraph.parquet"),
                            rawfpath=os.path.join(feature_dir, "ipscore_raw.csv"))

[Info]====== Generate IP Graph Features for date: 2021-03-30 ======
[Info] Raw Data shape: (100, 2)
[Info] IP features: (100, 19)
[Info] Graph IP Features Save Data to: dummydata/features/2021-03-30/features_ipgraph.parquet


In [13]:
len2mal = gen_len2mal_score(logday, datafpath=data_fpath,
                            savefpath=os.path.join(feature_dir, "features_len2mal.parquet"),
                            histmalfpath="dummydata/graphtmp/malicious_hist.csv",
                            domscore_fpath=os.path.join(feature_dir, "domscore_raw.csv"),
                            ipscore_fpath=os.path.join(feature_dir, "ipscore_raw.csv"))

[Info]====== Generate Graph Connection Features for date: 2021-03-30 ======
[Info] Read historical malicious data: dummydata/graphtmp/malicious_hist.csv
[Info] Read Raw Data from:  dummydata/graphtmp/data_2021-03-30.csv
[Info] Raw Data shape: (100, 3)
[Info] Features Shape (100, 4)
[Info] Graph Connection Features Save Data to: dummydata/features/2021-03-30/features_len2mal.parquet


### Now we concat all features (graph and non-graph)

In [14]:
features_fmt = "features_{}.parquet"
features_typs = ["domgraph", "ipgraph", "histmal", "hist",
                 "fqdn", "per", "len2mal"]
drop_col = ['domain', 'id_resp_h', 'true_periods', 'total_maleng']

In [15]:
datadf = pd.DataFrame()

for feat_typ in features_typs:
    feat_fpath = os.path.join(feature_dir, features_fmt.format(feat_typ))
    print("reading features {} from: ".format(feat_typ), feat_fpath)
    
    tmpfeat = pd.read_parquet(feat_fpath).fillna(0)
    print(tmpfeat.columns)
    
    for col in tmpfeat.columns:
        if col in drop_col:
            tmpfeat = tmpfeat.drop(columns=col)
            print("drop", col)
    
    if len(datadf.columns) == 0:
        datadf = tmpfeat
    else:
        datadf = datadf.merge(tmpfeat, on="host", how="left")
        
        
datadf.to_parquet("dummydata/features/2021-03-30/features_{}.parquet".format(logday))

reading features domgraph from:  dummydata/features/2021-03-30/features_domgraph.parquet
Index(['host', 'domain', 'cntMalFQDNs', 'cntIP', 'cntFQDN', 'sumMalEng',
       'maxMalEng', 'avgMalEng', 'minMalEng', 'maxCisco', 'minCisco',
       'avgCisco', 'sumMal', 'maxMal', 'minMal', 'avgMal', 'malFQDN_ratio',
       'malEng_ratio'],
      dtype='object')
drop domain
reading features ipgraph from:  dummydata/features/2021-03-30/features_ipgraph.parquet
Index(['host', 'sum_ipDom', 'sum_ipMalDom', 'avg_ipDom', 'avg_ipMalDom',
       'max_ipDom', 'max_ipMalDom', 'min_ipDom', 'min_ipMalDom',
       'max_ipMalDomRatio', 'max_ipDomMalEngRatio', 'min_ipMalDomRatio',
       'min_ipDomMalEngRatio', 'avg_ipMalDomRatio', 'avg_ipDomMalEngRatio',
       'sum_ipDomMalEng', 'max_ipDomMalEng', 'min_ipDomMalEng',
       'avg_ipDomMalEng'],
      dtype='object')
reading features histmal from:  dummydata/features/2021-03-30/features_histmal.parquet
Index(['host', 'hist_malscore_min_period', 'hist_malscore_ma

In [18]:
datadf.columns

Index(['host', 'cntMalFQDNs', 'cntIP', 'cntFQDN', 'sumMalEng', 'maxMalEng',
       'avgMalEng', 'minMalEng', 'maxCisco', 'minCisco', 'avgCisco', 'sumMal',
       'maxMal', 'minMal', 'avgMal', 'malFQDN_ratio', 'malEng_ratio',
       'sum_ipDom', 'sum_ipMalDom', 'avg_ipDom', 'avg_ipMalDom', 'max_ipDom',
       'max_ipMalDom', 'min_ipDom', 'min_ipMalDom', 'max_ipMalDomRatio',
       'max_ipDomMalEngRatio', 'min_ipMalDomRatio', 'min_ipDomMalEngRatio',
       'avg_ipMalDomRatio', 'avg_ipDomMalEngRatio', 'sum_ipDomMalEng',
       'max_ipDomMalEng', 'min_ipDomMalEng', 'avg_ipDomMalEng',
       'hist_malscore_min_period', 'hist_malscore_max_period',
       'hist_malscore_mean_period', 'hist_malscore_median_period',
       'hist_malscore_ratio_period', 'freq', 'occ', 'psd_ratio', 'dom_illegal',
       'dom_sld_entropy', 'subdom_entropy', 'dom_entropy', 'fqdn_entropy',
       'dom_tldcnt', 'dom_sldcnt', 'dom_subcnt', 'dom_level', 'dom_length',
       'mean_fqdn_period', 'max_fqdn_period', 'min_f